# AFT model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sksurv.nonparametric import kaplan_meier_estimator
from lifelines import *
from lifelines.plotting import qq_plot
from lifelines.utils import find_best_parametric_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
def model_data():
    train_df = pd.read_csv(r'C:\Users\grina\Desktop\VGTU\final_data.csv', index_col=0)
    train_df.reset_index(inplace=True)
    #replace empty values with 0
    train_df.fillna(value=0, inplace=True)
    train_df.drop(columns=['name', 'Parachute'], inplace=True)
    #change T, F with 1,0
    train_df['is_in_blue_zone'] = train_df['is_in_blue_zone'].replace({True:1, False:0})
    train_df['is_in_red_zone'] = train_df['is_in_red_zone'].replace({True:1, False:0})
    train_df['event'] = 1
    return train_df

## Select parametric model

In [ ]:
df = model_data()
#Q-Q plot for different distributions
fig, axes = plt.subplots(2, 2, figsize=(20,20))
axes = axes.reshape(4,)
for i, model in enumerate([WeibullFitter(), LogNormalFitter(), LogLogisticFitter(), ExponentialFitter()]):
    model.fit(df['death_time'].div(60).round(2), df['event'])
    qq_plot(model, ax=axes[i])

#noone is clearly best choice

## Modelling LogLogisticAFT

In [ ]:
%%time
mms = MinMaxScaler(feature_range=(0, 10), copy=False)
df_aft = model_data()
df_aft = pd.concat([df_aft, pd.get_dummies(df_aft['playing_type'], prefix='playing_type')], axis=1)
df_aft.rename(columns={"playing_type_1":"solo", "playing_type_2":"duo", "playing_type_3":"squad"}, inplace=True)
#del multikolinearumo pasalinam duo playing_type_2 atributa
df_aft.drop(columns=['duo'], inplace=True)
#df_aft.loc[(df_aft['solo'] == 0) & (df_aft['squad'] == 0), 'squad'] = 1
df_aft.drop(columns=['distance_sum', 'index', 'playing_type', 'assist', 'item_stack_count', 'damage', 'dist_on_freefall','rank'], inplace=True)
not_scaled = ['event', 'solo', 'squad', 'groggy', 'is_in_blue_zone', 'is_in_red_zone', 'death_time']
df_scaled = df_aft.drop(columns=not_scaled)

#scaling features
scaled_features = mms.fit_transform(df_scaled.values)
df_scaled = pd.DataFrame(scaled_features, index=df_scaled.index, columns=df_scaled.columns)
df_scaled[not_scaled] = df_aft[not_scaled]
X_train, X_test, y_train, y_test = train_test_split(df_scaled, df_scaled['death_time'], test_size=0.2, random_state=20)

aft = LogLogisticAFTFitter(penalizer=5e-2, l1_ratio=0.5)
aft.fit(X_train, duration_col='death_time', event_col='event')
aft.print_summary()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax = aft.plot(['dist_on_foot', 'dist_on_vehicle', 'is_in_blue_zone', 'dist_on_swim', 'dist_on_parachute'])
plt.title('5 didžiausią įtaką pagreitintam įvykio laikui turintys atributai')
plt.xlabel('log(Pagreitintas įvykio laikas), (CI 95%)', fontsize=12)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 15))
ax = aft.plot(df_aft.columns.tolist())
plt.title('Pagreitinto įvykio laiko modelio atributų koeficientai')
plt.xlabel('log(Pagreitintas įvykio laikas), (CI 95%)', fontsize=12)

In [ ]:
df_ll = model_data()
df_ll = pd.concat([df_ll, pd.get_dummies(df_ll['playing_type'], prefix='playing_type')], axis=1)
#del multikolinearumo pasalinam duo playing_type_2 atributa
df_ll.drop(columns=['playing_type_2'], inplace=True)
df_ll.drop(columns=['distance_sum', 'index', 'playing_type', 'assist', 'item_stack_count', 'damage', 'dist_on_freefall', 'is_in_red_zone', 'playing_type_3', 'rank'], inplace=True)

llf = LogLogisticAFTFitter(penalizer=0.05).fit(df_ll, 'death_time', 'event')
lnf = LogNormalAFTFitter(penalizer=0.05).fit(df_ll, 'death_time', 'event')
wf = WeibullAFTFitter(penalizer=0.05).fit(df_ll, 'death_time', 'event')
print(llf.log_likelihood_) #best choice, biggest log-likelihood
print(lnf.log_likelihood_) 
print(wf.log_likelihood_)

## PySurvival LogLogisticAFT 

In [ ]:
from pysurvival.models.parametric import ExponentialModel
from pysurvival.models.parametric import LogNormalModel
from pysurvival.models.parametric import WeibullModel
from pysurvival.models.parametric import LogLogisticModel
from pysurvival.models.parametric import GompertzModel
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.display import integrated_brier_score
from sklearn.model_selection import train_test_split
from pysurvival.utils.display import display_loss_values
import torch

In [ ]:
df_pysurvival_aft = model_data()
df_pysurvival_aft = pd.concat([df_pysurvival_aft, pd.get_dummies(df_pysurvival_aft['playing_type'], prefix='playing_type')], axis=1)
#del multikolinearumo pasalinam duo playing_type_2 atributa
df_pysurvival_aft.drop(columns=['playing_type_2'], inplace=True)
df_pysurvival_aft.loc[(df_pysurvival_aft['playing_type_1'] == 0) & (df_pysurvival_aft['playing_type_3'] == 0), 'playing_type_3'] = 1
df_pysurvival_aft['death_time'] = df_pysurvival_aft['death_time'].div(60).round(0)
df_pysurvival_aft.drop(columns=['distance_sum', 'index', 'playing_type', 'assist', 'item_stack_count', 'damage', 'dist_on_freefall', 'playing_type_3', 'rank'], inplace=True)
index_train, index_test = train_test_split(range(df_pysurvival_aft.shape[0]), test_size = 0.2, random_state=20)
data_train = df_pysurvival_aft.loc[index_train].reset_index(drop=True)
data_test  = df_pysurvival_aft.loc[index_test].reset_index(drop=True)
X_train, X_test = data_train.drop(columns=['death_time', 'event']), data_test.drop(columns=['death_time', 'event'])
T_train, T_test = data_train['death_time'].values, data_test['death_time'].values
E_train, E_test = data_train['event'].values, data_test['event'].values

In [ ]:
%%time
ll_model = LogLogisticModel()
#ln_model.fit(X_train, T_train, E_train, init_method='zeros', lr=3.62e-4, num_epochs=2000, optimizer='adam')
ll_model.fit(X_train, T_train, E_train, init_method='glorot_uniform', lr=3.6e-4, num_epochs=2000, optimizer='adam')
#ln_model.fit(X_train, T_train, E_train, init_method='glorot_normal', lr=3.42e-4, num_epochs=2000, optimizer='adam')
display_loss_values(ll_model)

In [ ]:
#### 5 - Cross Validation / Model Performances / C-INDEX
c_index = concordance_index(ll_model, X_test, T_test, E_test)
print('C-index: {:.2f}'.format(c_index))

In [ ]:
prediction = ln_model.predict_risk(X_test)

## sksurv IPCRidge AFT model

In [ ]:
from sksurv.linear_model import IPCRidge
from sklearn.model_selection import train_test_split

df_aft_sksurv = model_data()
df_aft_sksurv = pd.concat([df_aft_sksurv, pd.get_dummies(df_aft_sksurv['playing_type'], prefix='playing_type')], axis=1)
df_aft_sksurv.rename(columns={"playing_type_1":"solo", "playing_type_2":"duo", "playing_type_3":"squad"}, inplace=True)
#del multikolinearumo pasalinam duo playing_type_2 atributa
df_aft_sksurv.drop(columns=['duo'], inplace=True)
#df_aft_sksurv.loc[(df_aft_sksurv['solo'] == 0) & (df_aft_sksurv['squad'] == 0), 'squad'] = 1

df_aft_sksurv.drop(columns=['distance_sum', 'index', 'playing_type', 'assist', 'item_stack_count', 'damage', 'dist_on_freefall', 'rank'], inplace=True)
#df_aft_sksurv['death_time'] = df_aft_sksurv['death_time'].div(60).round(3)
Xt = df_aft_sksurv.drop(columns=['death_time', 'event'])
y = df_aft_sksurv[['event', 'death_time']]
y['event'] = y['event'].astype('bool')
s = y.dtypes
yt = np.array([tuple(x) for x in y.values], dtype=list(zip(s.index, s)))

X_train, X_test, y_train, y_test = train_test_split(Xt, yt, test_size=0.3, random_state=20)

aft_sksurv = IPCRidge(alpha=0.1)
aft_sksurv.fit(X_train, y_train)

In [ ]:
def fit_and_score_features(X, y):
    n_features = X.shape[1]
    scores = np.empty(n_features)
    m = IPCRidge(alpha=0.1)
    for j in range(n_features):
        Xj = X[:, j:j+1]
        m.fit(Xj, y)
        scores[j] = m.score(-Xj, y)
    return scores

scores = fit_and_score_features(X_train.values, y_train)
pd.Series(scores, index=X_train.columns).sort_values(ascending=False)

In [ ]:
train_min, train_max = y_train['death_time'].min(), y_train['death_time'].max()
test_min, test_max = y_test['death_time'].min(), y_test["death_time"].max()

In [ ]:
from sksurv.metrics import (concordance_index_censored,concordance_index_ipcw,cumulative_dynamic_auc)
pred_aft = aft_sksurv.predict(X_test)
res_c = concordance_index_censored(y_test['event'], y_test['death_time'], -np.log(pred_aft))
res_ipcw = concordance_index_ipcw(y_train, y_test, -np.log(pred_aft), tau=times[-1])
res_c[0]
res_ipcw[0]

In [ ]:
times = np.arange(1.5, 31, 1)
va_auc, va_mean_auc = cumulative_dynamic_auc(y_train, y_test, -np.log(pred_aft), times)
plt.plot(times, va_auc, marker="o")
plt.xlabel("Playing time (min)")
plt.axhline(va_mean_auc, linestyle="--")
plt.ylabel("AUC")
plt.grid(True)
va_mean_auc

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import median_absolute_error
from pysurvival import utils
from pysurvival.models.non_parametric import KaplanMeierModel
from pysurvival import utils
from pysurvival.utils import metrics

def act_to_est(model, X, T, E, figure_size, times = None,  metrics = ['rmse', 'mean', 'median']):
    kmf = KaplanMeierModel()
    kmf.fit(T, E)
    N = T.shape[0]
    if times is None:
        times = kmf.times
    actual = []
    predicted = []

    model_pred =  np.sum(model.predict_density(X), 0)
    for t in times:
        min_index = [abs(aj1-t) for (aj1, aj) in model.time_buckets]
        index = np.argmin(min_index)
        actual.append(N*kmf.predict_density(X,t))
        predicted.append(model_pred[index])

    results = None
    title = 'Realus ir prognozuojamas ivykių skaičius'
    if metrics is not None:
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        med_ae = median_absolute_error(actual, predicted)
        mae = mean_absolute_error(actual, predicted)

        #jei ivertinimo reikia tik vieno
        if isinstance(metrics, str) :
            if 'rmse' in metrics.lower() or 'root' in metrics.lower():
                results = rmse
                title += "\n Šaknis iš vidutinės kvadratinės paklaidos = {:.3f}".format(rmse)
            elif 'median' in metrics.lower() :
                results = med_ae
                title += "\n Absoliutinės paklaidos mediana = {:.3f}".format(med_ae)
            elif 'mean' in metrics.lower() :
                results = mae
                title += "\n Vidutinė absoliutinė paklaida = {:.3f}".format(mae)
            else:
                raise NotImplementedError('{} nėra tokio įvertinimo'.format(metrics))

        #jei reikalingu ivertinimu reikia saraso
        elif isinstance(metrics, list):
            results = {}
            is_rmse = False
            if any( [ ('rmse' in m.lower() or 'root' in m.lower()) \
                for m in metrics ]):
                is_rmse = True
                results['rmse'] = rmse
                title += "\n Šaknis iš vidutinės kvadratinės paklaidos = {:.3f}".format(rmse)
            is_med_ae = False
            if any( ['median' in m.lower() for m in metrics ]):
                is_med_ae = True
                results['median'] = med_ae
                title += "\n Absoliutinių paklaidų mediana = {:.3f}".format(med_ae)
            is_mae = False
            if any( ['mean' in m.lower() for m in metrics ]):
                is_mae = True
                results['mean'] = mae
                title += "\n Vidutinė absoliutinė paklaida = {:.3f}".format(mae)
            if all([not is_mae, not is_rmse, not is_med_ae]):
                error = 'Nurodyti vertinimai nerasti'
                raise NotImplementedError(error)

    fig, ax = plt.subplots(figsize=figure_size)
    ax.plot(times, actual, color='red', label='Realus', alpha=0.8, lw = 3)
    ax.plot(times, predicted, color='blue', label='Prognozuojamas', alpha=0.8, lw = 3)
    plt.xlim(0, max(T))
    ax.set_ylim(0)
    plt.xlabel('Laikas (min)', fontsize=13)
    plt.ylabel('Įvykių skaičius', fontsize=13)
    plt.title(title, fontsize = 15)
    plt.legend(fontsize = 15)
    plt.show()

    return results

In [ ]:
results_end = act_to_est(ll_model, X_test, T_test, E_test, figure_size=(15, 6), metrics=['rmse', 'mean', 'median'])

In [ ]:
from pysurvival.utils.metrics import brier_score
def brier_score_plot(model, X, T, E, figure_size):
    
    times, brier_scores = brier_score(model, X, T, E)
    times.insert(0, 0)
    brier_scores.insert(0, 0)
    ibs_value = np.trapz(brier_scores, times)/max(T)

    fig, ax = plt.subplots(figsize=figure_size)
    title = 'Brier įvertinimų vidurkis = {:.2f}'
    title = title.format(ibs_value)
    ax.axhline(y=0.25, ls = 'dotted', color = 'red')
    ax.plot(times, brier_scores, color = 'blue', lw = 2)
    ax.set_xlim(0, max(T))
    ax.set_ylim(0)
    plt.xlabel('Laikas (min)', fontsize=13)
    plt.ylabel('Brier įvertinimas BS(t)', fontsize=13)
    ax.axhline(y=0.25, ls = 'dotted', color = 'red')
    plt.title(title, fontsize=18)
    plt.show()
    return ibs_value

In [ ]:
ibs = brier_score_plot(ll_model, X_test, T_test, E_test, figure_size=(15, 6))